In [1]:
import earthaccess
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import google.generativeai as genai
import os

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Carregar variaveis de ambiente
load_dotenv()

True

In [3]:
# Vai abrir prompt para login (usuário e senha do Earthdata)
auth = earthaccess.login(strategy="environment")

In [4]:
def get_history_json(hour: int, day: int, month: int, lat: float, lon: float):
    """
    Busca o mesmo horário/dia/mês nos últimos 5 anos válidos (respeitando o limite 2025-06)
    e retorna JSON compacto com as principais variáveis meteorológicas convertidas.
    Inclui precipitação de neve.
    """
    start_year = 2025 if month < 6 else 2024
    years = list(range(start_year, start_year - 5, -1))  # ex: [2025, 2024, 2023, 2022, 2021]

    fs = earthaccess.get_fsspec_https_session()
    output = []

    # Variáveis de interesse e conversões
    vars_map = {
        "temperatura_ar": ("Tair_f_inst", lambda v: v - 273.15),         # K → °C
        "precipitacao": ("Rainf_f_tavg", lambda v: v * 3600),           # kg/m²/s → mm/h
        "neve": ("Snowf_tavg", lambda v: v * 3600),                     # kg/m²/s → mm/h
        "vento": ("Wind_f_inst", lambda v: v),                          # m/s
        "radiacao_solar": ("SWdown_f_tavg", lambda v: v),               # W/m²
        "umidade_do_ar": ("Qair_f_inst", lambda v: v * 1000),           # kg/kg → g/kg
        "umidade_do_solo": ("SoilMoi0_10cm_inst", lambda v: v),         # kg/m²
    }

    def get_scalar(ds, var):
        try:
            return float(np.asarray(ds[var].values).squeeze())
        except Exception:
            return np.nan

    for year in years:
        try:
            start_dt = datetime(year, month, day, hour)
            end_dt = start_dt + timedelta(hours=2)

            results = earthaccess.search_data(
                short_name="GLDAS_NOAH025_3H",
                version="2.1",
                temporal=(start_dt.isoformat(), end_dt.isoformat())
            )
            if not results:
                print(f"⚠️ Nenhum arquivo em {year}-{month:02d}-{day:02d} {hour:02d}h")
                continue

            ds = xr.open_dataset(fs.open(results[0].data_links()[0], mode="rb"))
            p = ds.sel(lat=lat, lon=lon, method="nearest")

            rec = {"year": year}
            for key, (var, conv) in vars_map.items():
                val = get_scalar(p, var)
                rec[key] = round(conv(val), 3) if np.isfinite(val) else None

            output.append(rec)
            print(f"✅ {year}: dados coletados")

        except Exception as e:
            print(f"❌ Erro {year}: {e}")
            continue

    if not output:
        raise ValueError("Nenhum dado retornado para o histórico solicitado.")

    return {
        "lat": float(p.lat.values),
        "lon": float(p.lon.values),
        "dados": output
    }


In [9]:
data_json = get_history_json(hour=12, day=5, month=10, lat=-22.90, lon=-43.20)

✅ 2024: dados coletados
✅ 2023: dados coletados
✅ 2022: dados coletados
✅ 2021: dados coletados
✅ 2020: dados coletados


In [10]:
df = pd.DataFrame(data_json["dados"])
df

,year,temperatura_ar,precipitacao,neve,vento,radiacao_solar,umidade_do_ar,umidade_do_solo
0,2024,20.170,0.000,0.0,3.156,323.479,11.368,30.950
1,2023,25.783,0.419,0.0,0.504,161.660,16.513,38.571
2,2022,20.642,0.000,0.0,1.107,198.370,13.480,35.340
3,2021,22.052,1.079,0.0,1.502,165.170,14.828,38.695
4,2020,25.236,0.000,0.0,1.805,227.050,14.220,33.444


In [11]:
def compute_weather_stats(history_json: dict,
                                          rain_thresh: float = 0.1,
                                          snow_thresh: float = 0.1):
    """
    Calcula médias e probabilidades usando limiares:
      - probabilidade de chuva = fração de anos com precipitacao >= rain_thresh (mm/h)
      - probabilidade de neve  = fração de anos com neve        >= snow_thresh (mm/h)
    Para variáveis contínuas (temp, vento, radiação, umidade_ar, umidade_solo),
    só retorna média (prob = None).
    """
    df = pd.DataFrame(history_json["dados"])
    stats = {}
    total = len(df)

    def mean_or_none(series):
        s = series.dropna()
        return round(float(s.mean()), 3) if len(s) else None

    # --- MÉDIAS ---
    stats["temperatura_ar"] = {"media": mean_or_none(df.get("temperatura_ar", pd.Series(dtype=float))), "prob": None}
    stats["vento"]          = {"media": mean_or_none(df.get("vento", pd.Series(dtype=float))),           "prob": None}
    stats["radiacao_solar"] = {"media": mean_or_none(df.get("radiacao_solar", pd.Series(dtype=float))), "prob": None}
    stats["umidade_do_ar"]  = {"media": mean_or_none(df.get("umidade_do_ar", pd.Series(dtype=float))),  "prob": None}
    stats["umidade_do_solo"]= {"media": mean_or_none(df.get("umidade_do_solo", pd.Series(dtype=float))),"prob": None}

    # --- PROBABILIDADES COM LIMIAR ---
    # Chuva
    rain_series = df.get("precipitacao", pd.Series(dtype=float)).dropna()
    rain_hits = int((rain_series >= rain_thresh).sum()) if len(rain_series) else 0
    rain_prob = round(rain_hits / total, 2) if total else None
    stats["precipitacao"] = {
        "media": mean_or_none(rain_series),
        "prob": rain_prob,
        "ocorrencias": rain_hits,
        "amostras": total,
        "limiar_mm_h": rain_thresh
    }

    # Neve
    snow_series = df.get("neve", pd.Series(dtype=float)).dropna()
    snow_hits = int((snow_series >= snow_thresh).sum()) if len(snow_series) else 0
    snow_prob = round(snow_hits / total, 2) if total else None
    stats["neve"] = {
        "media": mean_or_none(snow_series),
        "prob": snow_prob,
        "ocorrencias": snow_hits,
        "amostras": total,
        "limiar_mm_h": snow_thresh
    }

    return {
        "lat": history_json["lat"],
        "lon": history_json["lon"],
        "estatisticas": stats
    }


In [12]:
compute_weather_stats(data_json)

{'lat': -22.875,
 'lon': -43.125,
 'estatisticas': {'temperatura_ar': {'media': 22.777, 'prob': None},
  'vento': {'media': 1.615, 'prob': None},
  'radiacao_solar': {'media': 215.146, 'prob': None},
  'umidade_do_ar': {'media': 14.082, 'prob': None},
  'umidade_do_solo': {'media': 35.4, 'prob': None},
  'precipitacao': {'media': 0.3,
   'prob': 0.4,
   'ocorrencias': 2,
   'amostras': 5,
   'limiar_mm_h': 0.1},
  'neve': {'media': 0.0,
   'prob': 0.0,
   'ocorrencias': 0,
   'amostras': 5,
   'limiar_mm_h': 0.1}}}

In [13]:
def summarize_weather(stats_json):
    """
    Gera um resumo textual geral das condições climáticas com base nas médias e probabilidades
    do histórico, incluindo as unidades de medida para melhor contexto físico.
    """
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    model = genai.GenerativeModel("gemini-2.5-flash")

    # --- monta prompt humanizado e descritivo ---
    prompt = f"""
    Gere um resumo breve (2 a 3 frases) com linguagem leve e humana,
    descrevendo as condições climáticas gerais com base nas médias e probabilidades abaixo.

    O texto deve soar como uma previsão do tempo cotidiana,
    sem mencionar coordenadas geográficas nem termos técnicos.

    Unidades das variáveis:
    - temperatura_ar: °C
    - precipitacao e neve: mm/h
    - vento: m/s
    - radiacao_solar: W/m²
    - umidade_do_ar: g/kg
    - umidade_do_solo: kg/m²

    Dados:
    {stats_json['estatisticas']}

    Instruções:
    - Fale de forma natural e positiva, como um boletim informal.
    - Resuma temperatura, chuva, vento e umidade de modo compreensível.
    - Se a chance de chuva for baixa, diga que o tempo está bom para atividades ao ar livre.
    - Se for alta, comente sobre possibilidade de chuva e clima úmido.
    - Use tom acolhedor, fluido e humano.
    """

    response = model.generate_content(prompt)
    return response.text.strip()


In [14]:
summarize_weather(compute_weather_stats(data_json))

'O dia promete ser super agradável, com temperaturas amenas por volta dos 18 graus e uma brisa leve refrescando o ambiente. A chance de chuva é bem pequena, então aproveite para curtir o tempo ao ar livre, com o ar um pouco mais sequinho!'